In [1]:
%pwd

'c:\\Users\\hp\\Documents\\Python_vs\\venv\\Proj_chatbot\\Medical_chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Documents\\Python_vs\\venv\\Proj_chatbot\\Medical_chatbot'

In [10]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables
os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API_KEY")

In [12]:
#Extract data from the pdf files
def load_pdf_files(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [13]:

extracted_data=load_pdf_files(data='Data')

In [14]:
def text_split(extraxted_data):
    text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_split.split_documents(extraxted_data)
    return text_chunks

In [15]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

5860


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
def download_huggingface_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings=download_huggingface_face_embeddings()

In [21]:
query_result=embeddings.embed_query("Hello world")
len(query_result)

384

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

In [23]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4X1GkE_Ny6a48MuM5AsEWndXXRqUyT5WwxYB7T9cHosCr9fqvNNeyGFnmrFTmWxK2fGKjz")

In [ ]:
index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [27]:
#load existing index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [28]:
docsearch

In [29]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})
retrieved_docs=retriever.invoke("what is acne?")

In [30]:
retrieved_docs

[Document(id='6c59d07e-0841-44d4-b88c-ae339d9d8710', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='eb32d689-5836-4b77-9dea-410aa7ace98d', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='e1c8ed48-efc7-4a23-91f6-1c0c59d2789a', metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when

In [31]:
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain #Cobine all documents & send it to the context
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain #The Retrieval Chain acts as a link, connecting queries and relevant documents before presenting them to the LLM for response generation

In [33]:
system_prompt=(
    "You are an assistant for question-answering task."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentence maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"

)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)


In [34]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [36]:
response=rag_chain.invoke({"input":"what is acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, caused when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
